## Importing necessary libraries

In [15]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.linear_model import LogisticRegression

## Fetching the datafile

In [2]:
df=pd.read_excel('titanic.xls')
original_df=pd.DataFrame.copy(df)

## Data preparation

In [3]:
df.head()
# df.drop(['name','body'],1,inplace=True)
df.fillna(0,inplace=True)

In [22]:
# Pclass Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
# survival Survival (0 = No; 1 = Yes)
# name Name
# sex Sex
# age Age
# sibsp Number of Siblings/Spouses Aboard
# parch Number of Parents/Children Aboard
# ticket Ticket Number
# fare Passenger Fare (British pound)
# cabin Cabin
# embarked Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
# boat Lifeboat
# body Body Identification Number
# home.dest Home/Destination

In [4]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,0.0,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,0.0,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,0,0.0,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,0,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,0,0.0,"Montreal, PQ / Chesterville, ON"


In [5]:
def handle_non_numeric_data(df):
    columns=df.columns.values
    for column in columns:
        text_digit_values={}
        def cover_to_int(val):
            return text_digit_values[val]
        
        if df[column].dtype!=np.int64 and df[column].dtype!=np.float64:
            unique_column_contents=set(df[column].values.tolist())
            x=0
            for i in unique_column_contents:
               if i not in  text_digit_values:
                text_digit_values[i]=x
                x+=1

            df[column]=list(map(cover_to_int,df[column]))
            
    return df


In [6]:
df=handle_non_numeric_data(df)

In [7]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,294,1,29.0000,0,0,744,211.3375,26,2,2,0.0,301
1,1,1,932,0,0.9167,1,2,503,151.5500,89,2,13,0.0,104
2,1,0,15,1,2.0000,1,2,503,151.5500,89,2,0,0.0,104
3,1,0,1230,0,30.0000,1,2,503,151.5500,89,2,0,135.0,104
4,1,0,763,1,25.0000,1,2,503,151.5500,89,2,0,0.0,104


## Specifying the labels and features

In [8]:

y=np.array(df['survived'])
X=np.array(df.drop(['survived','ticket','boat'],1).astype(float))
clf=MeanShift()

C:\Users\HARSHP~1\AppData\Local\Temp/ipykernel_21984/2139087414.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X=np.array(df.drop(['survived','ticket','boat'],1).astype(float))


In [9]:
clf.fit(X)
X=preprocessing.scale(X)
labels=clf.labels_
cluster_centers=clf.cluster_centers_
original_df['cluster_grp']=np.nan

In [10]:
for i in range(len(X)):
    original_df['cluster_grp'].iloc[i]=labels[i]

c:\users\harshpreet kaur\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [11]:
clusters=len(np.unique(labels))
clusters

2

In [12]:
survival_rates={}
for i in range(clusters):
    temp_df=original_df.loc[original_df['cluster_grp']==float(i)]
    survival_df=temp_df[temp_df['survived']==1]
    survival_rate=len(survival_df)/len(temp_df)
    survival_rates[i]=survival_rate
survival_rates

{0: 0.4019138755980861, 1: 0.36363636363636365}

## Building a model and defining a classifier

In [17]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33)

In [18]:
# cfr=neighbors.KNeighborsClassifier()
# cfr=svm.SVC()
cfr=LogisticRegression()
# clf_knn=KNeighborsClassifier()
cfr.fit(X_train,y_train)
accuracy=cfr.score(X_test,y_test)
accuracy

0.8148148148148148